<a href="https://colab.research.google.com/github/jones-semicolon/sign-language-translator/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies

In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

Importing Dependencies

In [ ]:
from google.colab import files
from google.colab import drive
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/slt_dataset'

In [12]:
cp -rf "/content/drive/MyDrive/asl_gesture/none/" "American/"

Upload the dataset zip file

In [ ]:
files.upload()

In [7]:
# !rmdir /content/asl_dataset/5/.ipynb_checkpoints/
dataset = "American" #change the data depends on the uploaded folder name
#!unzip data + ".zip"

Read the dataset

In [16]:
print(dataset)
labels = []
for i in os.listdir(dataset):
  if os.path.isdir(os.path.join(dataset, i)):
    labels.append(i)
print(labels)
if labels.count:
  dataset_path = 'American'

American
['l', 'r', 'p', 's', '8', 'v', 'z', 'c', '9', '6', 'i', 'j', 'x', 'a', 'q', 'ILY', '1', '7', 'n', '0', 'w', 'b', 't', '3', 'o', '5', 'd', 'y', 'u', '4', '2', 'none', 'e', 'k', 'f', 'g', 'm', 'h']


Plotting a couple examples inside the dataset for verification

In [ ]:
from IPython.display import display, Image
NUM_EXAMPLES = 5 #Adjust based in the lowest length of every data

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')
display(fig)
# plt.close(fig)

Loading the dataset based on path

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Train the model

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 39)                5031      
 out (Dense)                                                     
                                                             

Evaluating model performance

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

404/404 [==============================] - 2s 1ms/step - loss: 0.1742 - categorical_accuracy: 0.9282
Test loss:0.17424598336219788, Test accuracy:0.9282178282737732


Exporting the model

In [ ]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [ ]:
files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>